In [ ]:
#!import Testing.dib

Installing Packages Expecto

Installing Packages Expecto.

Installing Packages Expecto..

Installing Packages Expecto...

Installing Packages Expecto....

...

Installing Packages Expecto..............

Installing Packages Expecto...............

Installing Packages Expecto................

Installing Packages Expecto.................

Installed Packages Expecto, 10.1.0

In [ ]:
#r "nuget:FParsec"

Installing Packages FParsec

Installing Packages FParsec.

Installing Packages FParsec..

Installing Packages FParsec...

Installing Packages FParsec....

...

Installing Packages FParsec.......

Installing Packages FParsec........

Installing Packages FParsec.........

Installing Packages FParsec..........

Installed Packages FParsec, 1.1.1

In [ ]:
open FParsec

In [ ]:
type Block =
    {
        magic : string
        content : string
    }

In [ ]:
let magicMarker : Parser<string, unit> = pstring "#!"

In [ ]:
//// test

let input = "#!magic"
let result = run magicMarker input
result |> _equal (
    Success ("#!", (), Position ("", 2, 1, 3))
)

Success
      Item1: #!
      Item2: <null>
      Item3: Position
        Index: 2
        Line: 1
        Column: 3
        StreamName: 


In [ ]:
//// test

let input = "##!magic"
let result = run magicMarker input
result |> _equal (
    Failure (
        $"Error in Ln: 1 Col: 1{nl}##!magic{nl}^{nl}Expecting: '#!'{nl}",
        ParserError (
            Position ("", 0, 1, 1),
            (),
            ErrorMessageList (ExpectedString "#!")
        ),
        ()
    )
)

Failure
      Item1: Error in Ln: 1 Col: 1
##!magic
^
Expecting: '#!'

      Item2: ParserError
        Position: Position
          Index: 0
          Line: 1
          Column: 1
          StreamName: 
        UserState: <null>
        Messages: ErrorMessageList
          Head: ExpectedString
            String: #!
            Type: ExpectedString
          Tail: <null>
      Item3: <null>


In [ ]:
let magicCommand =
    magicMarker
    >>. manyTill anyChar newline
    |>> (String.Concat >> String.trim)

In [ ]:
//// test

let input = """#!magic

a"""
let result = run magicCommand input
result |> _equal (
    Success ("magic", (), Position ("", 8, 2, 1))
)

Success
      Item1: magic
      Item2: <null>
      Item3: Position
        Index: 8
        Line: 2
        Column: 1
        StreamName: 


In [ ]:
//// test

let input = """ #!magic

a"""
let result = run magicCommand input
result |> _equal (
    Failure (
        $"Error in Ln: 1 Col: 1{nl} #!magic{nl}^{nl}Expecting: '#!'{nl}",
        ParserError (
            Position ("", 0, 1, 1),
            (),
            ErrorMessageList (ExpectedString "#!")
        ),
        ()
    )
)

Failure
      Item1: Error in Ln: 1 Col: 1
 #!magic
^
Expecting: '#!'

      Item2: ParserError
        Position: Position
          Index: 0
          Line: 1
          Column: 1
          StreamName: 
        UserState: <null>
        Messages: ErrorMessageList
          Head: ExpectedString
            String: #!
            Type: ExpectedString
          Tail: <null>
      Item3: <null>


In [ ]:
let content =
    (newline >>. magicMarker) <|> (eof >>. preturn "")
    |> attempt
    |> lookAhead
    |> manyTill anyChar
    |>> (String.Concat >> String.trim)

In [ ]:
//// test

let input = """#!magic


a


"""
let result = run content input
result |> _equal (
    Success ("""#!magic


a""", (), Position ("", 14, 7, 1))
)

Success
      Item1: #!magic


a
      Item2: <null>
      Item3: Position
        Index: 14
        Line: 7
        Column: 1
        StreamName: 


In [ ]:
let block =
    pipe2
        magicCommand
        content
        (fun magic content ->
            {
                magic = magic
                content = content
            })

In [ ]:
//// test

let input = """#!magic


a


"""
let result = run block input
result |> _equal (
    Success (
        { magic = "magic"; content = "a" },
        (),
        Position ("", 14, 7, 1)
    )
)

Success
      Item1: Block
        magic: magic
        content: a
      Item2: <null>
      Item3: Position
        Index: 14
        Line: 7
        Column: 1
        StreamName: 


In [ ]:
let blocks =
    skipMany newline
    >>. sepEndBy block (skipMany1 newline)

In [ ]:
//// test

let input =
    """#!magic1

a

\#!magic2

b

""" |> escapeCell
let result = run blocks input
result |> _equal (
    Success (
        [
            { magic = "magic1"; content = "a" }
            { magic = "magic2"; content = "b" }
        ],
        (),
        Position ("", 26, 9, 1)
    )
)

Success
      Item1: FSharpList<Block>
        - magic: magic1
          content: a
        - magic: magic2
          content: b
      Item2: <null>
      Item3: Position
        Index: 26
        Line: 9
        Column: 1
        StreamName: 


In [ ]:
let formatBlock kernel (block : Block) =
    match kernel, block with
    | _, { magic = "markdown"; content = content } ->
        content.Split [| '\n' |]
        |> Array.map (fun line -> line.Trim ())
        |> Array.map (function
            | "" -> "//"
            | line -> $"// {line}"
        )
        |> String.concat "\n"
    | "fsharp", { magic = "fsharp"; content = content } when
        (content.StartsWith "//// test"
        || content.StartsWith "//// ignore")
        |> not ->
            content.Split [| '\n' |]
            |> Array.filter (fun line -> line.StartsWith "#r" |> not)
            |> String.concat "\n"
    | _ -> ""

In [ ]:
//// test

let input =
    """#!markdown


a

b


\#!markdown


c


\#!fsharp


let a = 1""" |> escapeCell
let result =
    match run block input with
    | Success (block, _, _) ->
        formatBlock "fsharp" block
    | Failure (msg, _, _) ->
        failwith msg
result |> _equal """// a
//
// b"""

// a
//
// b


In [ ]:
let formatBlocks kernel blocks =
    blocks
    |> List.map (formatBlock kernel)
    |> List.filter (fun s -> s <> "")
    |> String.concat "\n\n"

In [ ]:
//// test

let input =
    """#!markdown


a

b


\#!markdown


c


\#!fsharp


let a = 1""" |> escapeCell
let result =
    match run blocks input with
    | Success (blocks, _, _) ->
        formatBlocks "fsharp" blocks
    | Failure (msg, _, _) ->
        failwith msg
result |> _equal """// a
//
// b

// c

let a = 1"""

// a
//
// b

// c

let a = 1


In [ ]:
let run input =
    match run blocks input with
    | Success (result, _, _) -> Result.Ok result
    | Failure (errorMsg, _, _) -> Result.Error errorMsg

In [ ]:
let parseDibCode kernel file =
    let input = File.ReadAllText file
    match run input with
    | Result.Ok blocks ->
        blocks
        |> List.filter (fun block -> block.magic = kernel || block.magic = "markdown")
        |> formatBlocks kernel
    | Result.Error msg -> failwith msg

In [ ]:
let writeDibCode kernel file =
    printfn $"Parsing {file}"
    let output = parseDibCode kernel file
    let outputFileName =
        match kernel with
        | "fsharp" -> file.Replace (".dib", ".fs")
        | _ -> failwith "Unknown kernel"
    File.WriteAllText (outputFileName, output)

In [ ]:
//// ignore

let example1 =
    """#!meta

{"kernelInfo":{"defaultKernelName":"fsharp","items":[{"aliases":[],"name":"fsharp"},{"aliases":[],"name":"fsharp"}]}}

\#!fsharp

##r "nuget:Expecto"

\#!markdown

## ParserLibrary

\#!fsharp

open System

\#!markdown

### TextInput

\#!fsharp

type Position =
    {
        line : int
        column : int
    }"""
    |> escapeCell

printfn $"{example1}"

let blocks1 =
    run example1
    |> Result.toOption
    |> Option.get
blocks1

index value 0 { magic = "meta"\n content =\n "{"kernelInfo":{"defaultKernelName":"fsharp","items":[{"aliases":[],"name":"fsharp"},{"aliases":[],"name":"fsharp"}]}}" } magic meta content {"kernelInfo":{"defaultKernelName":"fsharp","items":[{"aliases":[],"name":"fsharp"},{"aliases":[],"name":"fsharp"}]}} 1 { magic = "fsharp"\n content = "##r "nuget:Expecto"" } magic fsharp content ##r "nuget:Expecto" 2 { magic = "markdown"\n content = "## ParserLibrary" } magic markdown content ## ParserLibrary 3 { magic = "fsharp"\n content = "open System" } magic fsharp content open System 4 { magic = "markdown"\n content = "### TextInput" } magic markdown content ### TextInput 5 { magic = "fsharp"\n content =\n "type Position =\n {\n line : int\n column : int\n }" } magic fsharp content type Position =
 {
 line : int
 column : int
 }

#!meta

{"kernelInfo":{"defaultKernelName":"fsharp","items":[{"aliases":[],"name":"fsharp"},{"aliases":[],"name":"fsharp"}]}}

#!fsharp

##r "nuget:Expecto"

#!markdown

## ParserLibrary

#!fsharp

open System

#!markdown

### TextInput

#!fsharp

type Position =
    {
        line : int
        column : int
    }


In [ ]:
//// test

blocks1
|> (formatBlocks "fsharp")
|> _equal """##r "nuget:Expecto"

// ## ParserLibrary

open System

// ### TextInput

type Position =
    {
        line : int
        column : int
    }"""

##r "nuget:Expecto"

// ## ParserLibrary

open System

// ### TextInput

type Position =
    {
        line : int
        column : int
    }


In [ ]:
//// test

blocks1
|> (formatBlocks "markdown")
|> _equal """// ## ParserLibrary

// ### TextInput"""

// ## ParserLibrary

// ### TextInput


In [ ]:
//// ignore

match Environment.GetEnvironmentVariable "OUTPUT" with
| "" | null -> ()
| path when System.IO.File.Exists path -> path |> writeDibCode "fsharp"
| path when path.Contains ";" -> path.Split [| ';' |] |> Array.iter (writeDibCode "fsharp")
| _ -> System.IO.Path.Combine (System.IO.Directory.GetCurrentDirectory (), "DibParser.dib") |> writeDibCode "fsharp"

Parsing Core.dib
Parsing Parser.dib
Parsing JsonParser.dib
Parsing DibParser.dib
